In [1]:
import ollama
from ollama import chat

from huggingface_hub import snapshot_download

from docling.datamodel.pipeline_options import PdfPipelineOptions, RapidOcrOptions
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.chunking import HybridChunker

from pydantic.v1 import BaseModel

from transformers import AutoTokenizer
from sentence_transformers import CrossEncoder

from BM25 import load_bm25, create_bm25, bm25_search
from util import docling_util
from util.text_splitter import RecursiveTextSplitter, DataFrameFormatter

import os
import json

/Users/yoyo/opt/anaconda3/envs/cathay_llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# document local path or URL

doc10 = "./test_doc/國泰醫院113年工作計畫.pdf" # cross page table and HAS index, scanned document OCR needed
doc11 = "./test_doc/國泰醫院113年經費預算.pdf" # large number table, scanned document OCR needed
doc12 = "./test_doc/國泰醫院112年工作報告.pdf" # table with check board inside, scanned document OCR needed
doc13 = "./test_doc/112年國泰醫療財團法人財務報告.pdf" # loose structured table, scanned document OCR needed
doc15 = "./test_doc/畢業學分審核表.pdf" # vertical table

# ollama embedding model
embed_model = "bge-m3:latest"

# reranker 
#rerank_model = CrossEncoder('BAAI/bge-reranker-large', max_length=512)
rerank_model = CrossEncoder('BAAI/bge-reranker-v2-m3', max_length=1024)

# ollama llm
#llm = "deepseek-r1:7b"
#llm = "deepseek-r1:14b"
#llm = "deepseek-r1:32b"
#llm = "deepseek-r1:14b-max-context"
#llm = "deepseek-r1:14b-quarter-context"
llm = "deepseek-r1:14b-eighth-context"

def get_embeddings(texts, model=embed_model):
    embed_response = ollama.embeddings(model=model, prompt=texts)
    embedded_vector = embed_response["embedding"]
    
    return embedded_vector

# PyPdfium without EasyOCR
# --------------------
"""pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = False
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = False
#pipeline_options.table_structure_options.do_cell_matching = True

#pipeline_options.table_structure_options.mode = 'accurate'

pipeline_options.images_scale = 2
pipeline_options.generate_page_images = True
pipeline_options.generate_picture_images = True

pypdfium_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(
        pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend
        )
    }
)"""

# PyPdfium with RapidOCR
# ----------------------
# Download RappidOCR models from HuggingFace
print("Downloading RapidOCR models")
download_path = snapshot_download(repo_id="SWHL/RapidOCR")

det_model_path = os.path.join(
    download_path, "PP-OCRv4", "ch_PP-OCRv4_det_infer.onnx"
)
rec_model_path = os.path.join(
    download_path, "PP-OCRv4", "ch_PP-OCRv4_rec_infer.onnx"
)
cls_model_path = os.path.join(
    download_path, "PP-OCRv3", "ch_ppocr_mobile_v2.0_cls_train.onnx"
)
ocr_options = RapidOcrOptions(
    det_model_path=det_model_path,
    rec_model_path=rec_model_path,
    cls_model_path=cls_model_path,
    #force_full_page_ocr=True
)

pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = True
pipeline_options.do_table_structure = True
#pipeline_options.table_structure_options.do_cell_matching = False
pipeline_options.table_structure_options.do_cell_matching = True
pipeline_options.table_structure_options.mode = 'accurate'

pipeline_options.images_scale = 2
pipeline_options.generate_page_images = True
pipeline_options.generate_picture_images = True

pipeline_options.ocr_options = ocr_options

pypdfium_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(
        pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend
        )
   }
)

# print json
def show_json(data):
    if isinstance(data, str):
        obj = json.loads(data)
        print(json.dumps(obj, indent=4, ensure_ascii=False))
    elif isinstance(data, dict) or isinstance(data, list):
        print(json.dumps(data, indent=4, ensure_ascii=False))
    elif issubclass(type(data), BaseModel):
        print(json.dumps(data.dict(), indent=4, ensure_ascii=False))

Fetching 21 files: 100%|██████████| 21/21 [00:00<00:00, 117440.51it/s]


# Docling reader, docling format parser output

In [25]:
#doc_source = [doc, doc1, doc2, doc3, doc4, doc5, doc6]
#doc_source = [doc4, doc5, doc7, doc8, doc9, doc10, doc11, doc12, doc13, doc14]
#doc_source = [doc10, doc11, doc12, doc13]                                      # scanned docs and loose table
doc_source = [doc15]

conv_results = pypdfium_converter.convert_all(
    doc_source,
    raises_on_error=True,  # to let conversion run through all and examine results at the end
)

conv_results_list = list(conv_results)

# Do hybrid chunking to merge similar chunk
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

hybrid_chunker = HybridChunker(
    tokenizer=tokenizer,
    max_tokens=8192,
    merge_peers=True  # optional, defaults to True
)

# text chunks
all_chunks = []
for conv_res in conv_results_list:
    docling_docs = conv_res.document
    chunk_iter = hybrid_chunker.chunk(dl_doc=docling_docs)
    chunks = list(chunk_iter)
    all_chunks += chunks


2025-04-16 11:27:12,918 - OrtInferSession - WARNING: DirectML is only supported in Windows OS. The current OS is Darwin. Use CoreMLExecutionProvider inference by default.
2025-04-16 11:27:13,000 - OrtInferSession - WARNING: DirectML is only supported in Windows OS. The current OS is Darwin. Use CoreMLExecutionProvider inference by default.
2025-04-16 11:27:13,045 - OrtInferSession - WARNING: DirectML is only supported in Windows OS. The current OS is Darwin. Use CoreMLExecutionProvider inference by default.


In [27]:
for i, chunk in enumerate(all_chunks[:]):
    print(f"=== {i} ===")
    txt_tokens = len(tokenizer.tokenize(chunk.text))
    print(f"chunk.text ({txt_tokens} tokens):\n{repr(chunk.text)}")

    ser_txt = hybrid_chunker.serialize(chunk=chunk)
    ser_tokens = len(tokenizer.tokenize(ser_txt))
    print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{repr(ser_txt)}")
    #print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{ser_txt}")

    print()

=== 0 ===
chunk.text (6117 tokens):
'學程：資訊科學系班级：資四甲學號：111016035姓名：萧大祐生日：911118\n列印日期：114/4/15\n列印時間：09：46：53\n業學分：133\n通識課程（已修18学分） （已修必修0学分，遥修18学,  = . 通識課程（已修18学分） （已修必修0学分，遥修18学, 學分成 = . 通識課程（已修18学分） （已修必修0学分，遥修18学, 科目名 = 基必修课程（已修58學分）. 通識課程（已修18学分） （已修必修0学分，遥修18学, 學分 = . 通識課程（已修18学分） （已修必修0学分，遥修18学, 成 = . 通識課程（已修18学分） （已修必修0学分，遥修18学, 科目名 = 門選機器學習. 通識課程（已修18学分） （已修必修0学分，遥修18学, 學分 = 3. 通識課程（已修18学分） （已修必修0学分，遥修18学, 成 = 90. 通識課程（已修18学分） （已修必修0学分，遥修18学, 科目名 = . 通識課程（已修18学分） （已修必修0学分，遥修18学, 學分成 = . 分),  = . 分), 學分成 = . 分), 科目名 = （已修必修58學分，選修0學分）. 分), 學分 = . 分), 成 = . 分), 科目名 = 门選體工程. 分), 學分 = 3. 分), 成 = 98. 分), 科目名 = . 分), 學分成 = . ,  = . , 學分成 = . , 科目名 = 门必演算法. , 學分 = 3. , 成 = 83. , 科目名 = 門選 硬體描述語言. , 學分 = 3. , 成 = 84. , 科目名 = . , 學分成 = . 领域課程-生涯職能领域(已修0,  = . 领域課程-生涯職能领域(已修0, 學分成 = . 领域課程-生涯職能领域(已修0, 科目名 = 門必計算機组. 领域課程-生涯職能领域(已修0, 學分 = 3. 领域課程-生涯職能领域(已修0, 成 = 77. 领域課程-生涯職能领域(已修0, 科目名 = 門選 企業系统分析與程式設計. 领域課程-生涯職能领域(已修0, 學分 = 3. 领域課程-生涯職能领域(已修0, 成 = 94. 领域課程-生涯職能领域(已修0, 科目名 = . 领域課程-生涯職能领

In [26]:
print(conv_results_list[0].document.export_to_markdown())

## 國立臺北教育大學學生业學分核表確單

學程：資訊科學系班级：資四甲學號：111016035姓名：萧大祐生日：911118

列印日期：114/4/15

列印時間：09：46：53

業學分：133

| 科目名                                              |    | 學分成   | 科目名                                              | 學分   | 成   | 科目名                        | 學分   | 成   | 科目名   | 學分成   |
|-----------------------------------------------------|----|----------|-----------------------------------------------------|--------|------|-------------------------------|--------|------|----------|----------|
| 通識課程（已修18学分） （已修必修0学分，遥修18学    |    |          | 基必修课程（已修58學分）                            |        |      | 門選機器學習                  | 3      | 90   |          |          |
| 分)                                                 |    |          | （已修必修58學分，選修0學分）                       |        |      | 门選體工程                    | 3      | 98   |          |          |
|                                                     |    |          | 门必演算法                                          | 3      |

# Rechunk large chunks with RecursiveTextSplitter

In [ ]:
big_chunk = []

for i, chunk in enumerate(all_chunks[:]):
    
    ser_txt = hybrid_chunker.serialize(chunk=chunk)
    ser_tokens = len(tokenizer.tokenize(ser_txt))
    if ser_tokens > 1024:
        print(f"=== {i} ===")
        #print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{repr(ser_txt)}")
        print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{ser_txt}")
        big_chunk.append(chunk)
        print()
    else:
        pass

In [ ]:
splitter = RecursiveTextSplitter(tokenizer=tokenizer, max_tokens=1024, overlap=150, min_length_ratio=1)

rechunked_large_chunks = []

for i, chunk in enumerate(big_chunk):
    ser_txt = hybrid_chunker.serialize(chunk=chunk)
    if splitter.tokenize_len(ser_txt) > splitter.max_tokens:
        sub_chunks = splitter.split_text(ser_txt)
        rechunked_large_chunks.extend(sub_chunks)
    else:
        rechunked_large_chunks.append(ser_txt)


"""ser_txt = hybrid_chunker.serialize(chunk=rechunked_large_chunks[1])
if splitter.tokenize_len(ser_txt) > splitter.max_tokens:
    sub_chunks = splitter.split_text(ser_txt)
    rechunked_large_chunks.extend(sub_chunks)
else:
    rechunked_large_chunks.append(ser_txt)"""

In [ ]:
for i, chunk in enumerate(rechunked_large_chunks[:]):
    
    ser_txt = chunk
    ser_tokens = len(tokenizer.tokenize(ser_txt))

    print(f"=== {i} ===")
    #print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{repr(ser_txt)}")
    print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{ser_txt}")
    print()

# Or table extraction result

In [ ]:
# table chunks
all_tables = docling_util.extract_tables(conv_results_list)

In [ ]:
table_formatter = DataFrameFormatter(tokenizer=tokenizer, show_index=False, max_tokens=1024)

table_chunks = []
for table in all_tables:
    chunks = table_formatter.chunk_rows(table)
    table_chunks.extend(chunks)

In [ ]:
for i, chunk in enumerate(table_chunks[:]):
    
    ser_txt = chunk
    ser_tokens = len(tokenizer.tokenize(ser_txt))

    print(f"=== {i} ===")
    #print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{repr(ser_txt)}")
    print(f"chunker.serialize(chunk) ({ser_tokens} tokens):\n{ser_txt}")
    print()

# Merge all chunks

In [ ]:
all_chunks.extend(rechunked_large_chunks)
all_chunks.extend(table_chunks)
all_chunks

In [ ]:
node_text, node_metadatas = [], []
for chunk in all_chunks:
    if type(chunk) == str:
        node_text.append(chunk)
        node_metadatas.append(None)
    else:
        node_text.append(hybrid_chunker.serialize(chunk=chunk))
        node_metadatas.append(chunk.meta.export_json_dict())

In [ ]:
embedded_text = []
for i in node_text:
    embedded_text.append(get_embeddings(i))

embedded_tables = []
for i in all_tables_chunks:
    embedded_tables.append(get_embeddings(i))

# Test on BM25 retrieval

In [ ]:
#query = "去新加坡出差可以申請多少費用？"
#query = "科主管國內出差可以申請多少餐雜宿費？"
query = "系統開發之安全管理，應包含哪些項目？"

output_dir = 'test_BM25index'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# as json
bm25_mixed_json = create_bm25(node_text, 'mixed')
bm25_mixed_json.save(os.path.join(output_dir, 'bm25_mixed.json'))

loaded_bm25_mixed_json = load_bm25(os.path.join(output_dir, 'bm25_mixed.json'), node_text)

print("load from json and mixure search:", query)
print()
results_json = loaded_bm25_mixed_json.search(query, top_k=5)
for doc_id, score in results_json:
    print(f"Doc ID: {doc_id}, Score: {score:.4f}, Text: {node_text[doc_id]}\n")

# Qdrant


In [3]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import PointStruct

class qdrant_DBConnector:
    def __init__(self, collection_name):#, embedding_fn):
        self.qdrant_client = QdrantClient("http://localhost:6333")

        self.collection_name = collection_name

        # create collection
        self.collection = self.qdrant_client.recreate_collection(
            collection_name = collection_name,
            vectors_config = models.VectorParams(
                distance = models.Distance.COSINE,
                size=len(get_embeddings("你好"))),
            optimizers_config = models.OptimizersConfigDiff(memmap_threshold=20000),
            hnsw_config = models.HnswConfigDiff(on_disk=True, m=16, ef_construct=100)
        )
        '''
        # embed models
        self.embedding_fn = embedding_fn
        qdrant_client.set_model(self.embedding_fn)
        '''

    ''' WARNING: NO CHECK ON EQUAL LENGTH YET'''
    def upsert_vector(self, vectors, data):
        # insert 'points' to qdrant by vector, 
        # payload with original text and metadata
        for i, vector in enumerate(vectors):
            """ WARNING: SHOULD CHECK DIMENSION==EMBEDDING_DIMENSION INSTEAD"""
            if len(vector) == 0:
                continue
            self.qdrant_client.upsert(
                collection_name=self.collection_name,
                points=[PointStruct(id=i,
                                    vector=vectors[i],
                                    payload={"text": data.text[i],
                                             "metadata": data.metadata[i]})]
                                    
            )

        print("upsert finish")

    def retrieved_all(self):
        count_points = self.qdrant_client.count(
            collection_name=self.collection_name,
            exact=True,
        )
        result = self.qdrant_client.retrieve(
            collection_name=self.collection_name,
            ids=list(range(0, count_points.count)),
        )
        return result

    def vector_search(self, vector, top_k):
        # vector search qdrant DB
        result = self.qdrant_client.search(
            collection_name=self.collection_name,
            query_vector=vector,
            limit=top_k,
            append_payload=True,
        )
        return result
    
    def vector_search_json(self, vector, top_k):
        # vector search qdrant DB with json format output
        result = self.qdrant_client.search(
            collection_name=self.collection_name,
            query_vector=vector,
            limit=top_k,
            append_payload=True,
        )
        vector_result_json = {
            f"chunk_{item.id}": {
                "text": item.payload['text'],
                "rank": index,
                "score": item.score
            }
            for index, item in enumerate(result)
        }

        return vector_result_json

In [ ]:
class DataObject:
    def __init__(self, text, metadata):
        self.text = text
        self.metadata = metadata if metadata else [{} for _ in range(len(text))]

data = DataObject(node_text, node_metadatas)
data.text[0]

In [ ]:
# create a qdrant vectorDB object
vector_db = qdrant_DBConnector("qdrant_test")
# add data to db
vector_db.upsert_vector(embedded_text, data)

table_datas = DataObject(all_tables_chunks, [])
# create a qdrant vectorDB object
vector_db_table = qdrant_DBConnector("qdrant_table")
# add data to db
vector_db_table.upsert_vector(embedded_tables, table_datas)

In [ ]:
#query = "國內出差費用標準"
#query = "國外出差，附表五 "
query = "系統開發之安全管理，應包含哪些項目？"

embedded_query = get_embeddings(query)
results = vector_db.vector_search(embedded_query, 5)
#results = vector_db_table.vector_search(embedded_query, 5)
print(f"尋找 「{query}」:")
#print(results[0])

for i, point in enumerate(results):
    print(f"\n🔹 Score: {point.score:.2f} - point ID: {point.id}:")
    print(point.payload['text'])
    print()


# Try chatting!

In [ ]:
"""stream = chat(
    model='deepseek-r1:7b',
    messages=[{'role': 'user', 'content': '妳好'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)"""

In [ ]:
def bm25_retrieval(query, vector_db_name=vector_db, top_k=3):
    result = vector_db_name.retrieved_all()
    retrieved_text = []
    for i, points in enumerate(result):
        retrieved_text.append(result[i].payload['text'])

    bm25_result = bm25_search(corpus=retrieved_text, query=query, top_k=top_k)
    bm25_result_json = {
        f"chunk_{item[0]}": {
            "text": item[2],
            "rank": index,
            "score": item[1]
        }
        for index, item in enumerate(bm25_result)
    }

    return bm25_result_json

def rrf(ranks, k=1):
    ret = {}
    # recursive through all retrieved method
    for rank in ranks:
        for id, val in rank.items():
            if id not in ret:
                ret[id] = {"score": 0, "text": val["text"]}
            # calculate rrf score
            ret[id]["score"] += 1.0/(k+val["rank"])
    # sort and return according to rrf score
    return dict(sorted(ret.items(), key=lambda item: item[1]["score"], reverse=True))

def get_completion(prompt, model=llm):
    messages = [{"role": "user", "content": prompt}]
    response = chat(
        model=model,
        messages=messages
        #stream=True,
        #format="json",
        #options={"temperature":0}
    )
    return response.message.content


In [ ]:
query = "去中國大陸出差可以申請多少費用？"

print("search for:", query)
bm25_retrieve_result = bm25_retrieval(query, top_k=3)
show_json(bm25_retrieve_result)

In [ ]:
embedded_query = get_embeddings(query)
vector_result_json = vector_db.vector_search_json(embedded_query, 3)
print("search for:", query)

show_json(vector_result_json)

In [ ]:
hybrid_result = rrf([vector_result_json, bm25_retrieve_result])

print(json.dumps(hybrid_result, indent=4, ensure_ascii=False))

In [ ]:
def hybrid_retriever(query, top_k=3):
    embedded_query = get_embeddings(query)
    result = rrf([vector_db.vector_search_json(embedded_query, top_k), bm25_retrieval(query, top_k=top_k)])
    return result

def reranker(query, retrieved_result, threshold=0):
    text_chunks = []
    for chunk_id, val in retrieved_result.items():
        text_chunks.append(val['text'])
    scores = rerank_model.predict([(query, doc) for doc in text_chunks])
    sorted_list = sorted(zip(scores, text_chunks), key=lambda x: x[0], reverse=True)
    reranked_result = [chunk for chunk in sorted_list if chunk[0] > threshold]
    if len(reranked_result) < 3:
        reranked_result = sorted_list[:3]
    elif len(reranked_result) > 5:
        reranked_result = reranked_result[:5]

    return reranked_result
    #return [chunk for chunk in sorted_list if chunk[0] > threshold]

def format_rag_output(reranked_list):
    formatted_docs = "\n\n".join([f"Document {i+1}:\n{text[1]}" for i, text in enumerate(reranked_list)])
    return formatted_docs

def hybrid_retriever_with_BM25table(query, top_k=3):
    embedded_query = get_embeddings(query)
    result = rrf([vector_db.vector_search_json(embedded_query, top_k), bm25_retrieval(query, top_k=top_k), bm25_retrieval(query, vector_db_table, top_k)])
    return result

In [ ]:
#query = "去南非共和國出差可以申請多少費用？"
query = "重要會計項目中，現金及約當現金的銀行存款是多少？"

#print(format_rag_output(reranker(query, hybrid_retriever(query, 20), 0.45)))
print(format_rag_output(reranker(query, hybrid_retriever(query, top_k=35), 0.45)))

print()
#for chunk in reranker(query, hybrid_retriever(query, 20)):
for chunk in reranker(query, hybrid_retriever(query, top_k=35)):
    print(chunk)
    print()

In [ ]:
# Job instruction
instruction = """
你是台灣國泰金控的聊天機器人秘書，專門為用戶解答公司內部的差旅費用、報銷標準等相關問題。
你的任務是根據你獲得的「參考文件」，對「用戶問題」段落的問題進行回答。

請務必根據「參考文件」中的具體資訊作答，並注意以下要求：
1. 若某些文件內容對回答無幫助，可以忽略，不採用。
2. 回答應簡潔、明確，避免冗長，僅提取關鍵資訊。
3. 若參考文件無法提供答案，請直接回答「我無法根據現有資料回答這個問題」，並不要自行補充。

嚴格使用繁體中文，避免英文或簡體中文。
"""

# User input
#input_text = "去中國大陸出差可以申請多少費用？"
#input_text = "去南非共和國出差可以申請多少費用？"
#input_text = "去智利出差可以申請多少費用？"
#input_text = "科主管國內出差可以申請多少餐雜宿費？"
#input_text = "商品貨幣是什麼？"
#input_text = "系統開發之安全管理，應包含哪些項目？"
#input_text = "國泰優惠APP 後台維護系統，有什麼功能？"
#input_text = "國泰優惠APP 後台維護系統，功能權限申請流程為何？" # Fail, didn't have the ability to recognize flow chart
#input_text = "國泰優惠 APP後台維護系統使用者權限表，有什麼欄位？"
#input_text = "金融科技創新業務之立案及概念測試流程為何？"  # Fail? didn't recognize flow chart but there's explaination on previous page
#input_text = "設計阿發時有什麼背景色彩限制？" # Fail? on picture detail explaination, but did catch the words on pics
#input_text = "根據111年度4-6月份捐款及捐贈物資徵信名冊，誰捐了鵝肉湯？" # doc14 fail on parsing compact table index, but seems like it does not interfere with llm answer
#input_text = "根據111年度4-6月份捐款及捐贈物資徵信名冊，梅力化學工業有限公司做了什麼？" # same as above
#input_text = "重要會計項目中，現金及約當現金的銀行存款，111年與112年分別是多少？" # loose structured table, success
#input_text = "幫我統整一下不動產、廠房及設備中，成本，折舊以及淨帳面金額" # loose structured table, poor recognize result, but seemingly success
#input_text = "請給我應收帳款(淨額)之帳齡分析的內容"
input_text = "請給我112年，藥品進貨交易對象的名稱，金額及比率"

# RAG retrieved documents
reranked_list = reranker(input_text, hybrid_retriever(input_text, 20), 0.45)
rag_docs = format_rag_output(reranked_list)

# Prompt template
prompt = f"""
# 任務
{instruction}

# 參考文件
{rag_docs}

# 用戶問題
{input_text}
"""

print("==== Prompt ====")
print(prompt)
print("================")

# llm calling
if len(rag_docs) != 0:
    response = get_completion(prompt)
else:
    response = "YAh, you retrieved NOTHING!"
print(response)

In [ ]:
text = """

==== Prompt ====

# 任務

你是台灣國泰金控的聊天機器人秘書，專門為用戶解答公司內部的差旅費用、報銷標準等相關問題。
你的任務是根據你獲得的「參考文件」，對「用戶問題」段落的問題進行回答。

請務必根據「參考文件」中的具體資訊作答，並注意以下要求：
1. 若某些文件內容對回答無幫助，可以忽略，不採用。
2. 回答應簡潔、明確，避免冗長，僅提取關鍵資訊。
3. 若參考文件無法提供答案，請直接回答「我無法根據現有資料回答這個問題」，並不要自行補充。

嚴格使用繁體中文，避免英文或簡體中文。


# 參考文件
Document 1:
民國 112 年度
單位：新台幣
1, 交易人名稱 = 裕利股份有限公司. 1, 金額 = $1,230,544,978. 1, 應付票據、  帳款餘額 = $538,625,956. 1, 占該項支  出之比率 = 45%. 2, 交易人名稱 = 台灣大昌華嘉股份有限公司. 2, 金額 = 425,136,734. 2, 應付票據、  帳款餘額 = 177,372,803. 2, 占該項支  出之比率 = 15%. 3, 交易人名稱 = 久裕企業股份有限公司. 3, 金額 = 163,077,219. 3, 應付票據、  帳款餘額 = 67,343,897. 3, 占該項支  出之比率 = 6%. 4, 交易人名稱 = 華安藥品股份有限公司. 4, 金額 = 66,678,740. 4, 應付票據、  帳款餘額 = 29,857,859. 4, 占該項支  出之比率 = 2%. 5, 交易人名稱 = 台灣東洋藥品工業股份有限公司. 5, 金額 = 45,251,402. 5, 應付票據、  帳款餘額 = 16,630,969. 5, 占該項支  出之比率 = 2%. 6, 交易人名稱 = 台田藥品股份有限公司. 6, 金額 = 44,433,615. 6, 應付票據、  帳款餘額 = 17,112,370. 6, 占該項支  出之比率 = 2%. 7, 交易人名稱 = 台灣中外製藥股份有限公司. 7, 金額 = 41,218,669. 7, 應付票據、  帳款餘額 = 15,854,114. 7, 占該項支  出之比率 = 1%. 8, 交易人名稱 = 瑞安國際股份有限公司. 8, 金額 = 31,866,988. 8, 應付票據、  帳款餘額 = 9,608,150. 8, 占該項支  出之比率 = 1%. 9, 交易人名稱 = 大隆興藥品股份有限公司. 9, 金額 = 29,255,417. 9, 應付票據、  帳款餘額 = 11,576,886. 9, 占該項支  出之比率 = 1%. 10, 交易人名稱 = 元英企業股份有限公司. 10, 金額 = 28,909,315. 10, 應付票據、  帳款餘額 = 9,644,946. 10, 占該項支  出之比率 = 1%

Document 2:
民國 111 年度
單位：新台幣
醫務收入, 交易對象 = 國泰人壽. 醫務收入, 關係 = 其他關係人. 醫務收入, 交易金額 = $9,318,908. 醫務收入, 授信期間 = 90 天. 醫務收入, 應收(付)票據、帳款.餘額 = $-. 醫務收入, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務收入, 備註 = 應收帳款. 醫務收入, 交易對象 = 國泰金控. 醫務收入, 關係 = 其他關係人. 醫務收入, 交易金額 = 781,800. 醫務收入, 授信期間 = 90 天. 醫務收入, 應收(付)票據、帳款.餘額 = -. 醫務收入, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務收入, 備註 = 應收帳款. 醫務收入, 交易對象 = 霖園公寓. 醫務收入, 關係 = 其他關係人. 醫務收入, 交易金額 = 208,000. 醫務收入, 授信期間 = 90 天. 醫務收入, 應收(付)票據、帳款.餘額 = -. 醫務收入, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務收入, 備註 = 應收帳款. 醫務收入, 交易對象 = 國泰世華. 醫務收入, 關係 = 其他關係人. 醫務收入, 交易金額 = 1,400,700. 醫務收入, 授信期間 = 90 天. 醫務收入, 應收(付)票據、帳款.餘額 = -. 醫務收入, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務收入, 備註 = 應收帳款. 醫務成本-  人事費用, 交易對象 = 國泰人壽. 醫務成本-  人事費用, 關係 = 其他關係人. 醫務成本-  人事費用, 交易金額 = 55,580,357. 醫務成本-  人事費用, 授信期間 = 60 天. 醫務成本-  人事費用, 應收(付)票據、帳款.餘額 = -. 醫務成本-  人事費用, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務成本-  人事費用, 備註 = 其他應付款. 醫務成本-  事務費用, 交易對象 = 國泰產險. 醫務成本-  事務費用, 關係 = 其他關係人. 醫務成本-  事務費用, 交易金額 = 4,229,871. 醫務成本-  事務費用, 授信期間 = 60 天. 醫務成本-  事務費用, 應收(付)票據、帳款.餘額 = -. 醫務成本-  事務費用, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務成本-  事務費用, 備註 = 其他應付款. 醫務成本-  事務費用, 交易對象 = 國泰人壽. 醫務成本-  事務費用, 關係 = 其他關係人. 醫務成本-  事務費用, 交易金額 = 3,144,672. 醫務成本-  事務費用, 授信期間 = 60 天. 醫務成本-  事務費用, 應收(付)票據、帳款.餘額 = -. 醫務成本-  事務費用, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務成本-  事務費用, 備註 = 其他應付款. 醫務成本-  事務費用, 交易對象 = 霖園公寓. 醫務成本-  事務費用, 關係 = 其他關係人. 醫務成本-  事務費用, 交易金額 = 5,427,619. 醫務成本-  事務費用, 授信期間 = 60 天. 醫務成本-  事務費用, 應收(付)票據、帳款.餘額 = 5,699,000. 醫務成本-  事務費用, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = 1.13%. 醫務成本-  事務費用, 備註 = 其他應付款. 醫務成本-  租金支出, 交易對象 = 國泰人壽. 醫務成本-  租金支出, 關係 = 其他關係人. 醫務成本-  租金支出, 交易金額 = 189,425,794. 醫務成本-  租金支出, 授信期間 = 不適用. 醫務成本-  租金支出, 應收(付)票據、帳款.餘額 = -. 醫務成本-  租金支出, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務成本-  租金支出, 備註 = 其他應付款. 其他營運費用, 交易對象 = 神坊資訊. 其他營運費用, 關係 = 其他關係人. 其他營運費用, 交易金額 = 6,942,134. 其他營運費用, 授信期間 = 60 天. 其他營運費用, 應收(付)票據、帳款.餘額 = 380,462. 其他營運費用, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = 0.08%. 其他營運費用, 備註 = 其他應付款

Document 3:
民國 111 年度
單位：新台幣
醫務收入, 交易對象 = 國泰人壽. 醫務收入, 關係 = 其他關係人. 醫務收入, 交易金額 = $9,318,908. 醫務收入, 授信期間 = 90 天. 醫務收入, 應收(付)票據、帳款.餘額 = $-. 醫務收入, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務收入, 備註 = 應收帳款. 醫務收入, 交易對象 = 國泰金控. 醫務收入, 關係 = 其他關係人. 醫務收入, 交易金額 = 781,800. 醫務收入, 授信期間 = 90 天. 醫務收入, 應收(付)票據、帳款.餘額 = -. 醫務收入, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務收入, 備註 = 應收帳款. 醫務收入, 交易對象 = 霖園公寓. 醫務收入, 關係 = 其他關係人. 醫務收入, 交易金額 = 208,000. 醫務收入, 授信期間 = 90 天. 醫務收入, 應收(付)票據、帳款.餘額 = -. 醫務收入, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務收入, 備註 = 應收帳款. 醫務收入, 交易對象 = 國泰世華. 醫務收入, 關係 = 其他關係人. 醫務收入, 交易金額 = 1,400,700. 醫務收入, 授信期間 = 90 天. 醫務收入, 應收(付)票據、帳款.餘額 = -. 醫務收入, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務收入, 備註 = 應收帳款. 醫務成本-  人事費用, 交易對象 = 國泰人壽. 醫務成本-  人事費用, 關係 = 其他關係人. 醫務成本-  人事費用, 交易金額 = 55,580,357. 醫務成本-  人事費用, 授信期間 = 60 天. 醫務成本-  人事費用, 應收(付)票據、帳款.餘額 = -. 醫務成本-  人事費用, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務成本-  人事費用, 備註 = 其他應付款. 醫務成本-  事務費用, 交易對象 = 國泰產險. 醫務成本-  事務費用, 關係 = 其他關係人. 醫務成本-  事務費用, 交易金額 = 4,229,871. 醫務成本-  事務費用, 授信期間 = 60 天. 醫務成本-  事務費用, 應收(付)票據、帳款.餘額 = -. 醫務成本-  事務費用, 應收(付)票據、帳款.占總應收(付)票  據帳款比率 = -%. 醫務成本-  事務費用, 備註 = 其他應付款. 醫務成本-  事務費用, 交易對象 = 國泰人壽. 醫務成本-  事務費用, 關係 = 其他關係人. 醫務成本-  事務費用, 交易金額 = 3,144,672. 醫務成本-  事務費用, 授信期間 = 60 天. 醫務成本-  事務費用, 應收(付)票據、帳款.餘額 = -. 醫務成本-  事務費用, 應收(付)票據、

# 用戶問題
請給我112年，藥品進貨交易對象的資訊

================
<think>
好的，我现在需要回答用户的问题：“請給我112年，藥品進貨交易對象的資訊”。根据任务要求，我要参考提供的三个文件来寻找相关信息。

首先，我查看了Document 1。这个文档标题是“民國 112 年度”，里面列出了多个交易人的名称、金額、應付票據、帳款餘額以及占該項支出的比率。看起来这些交易人主要是药品公司，比如裕利股份有限公司、台湾大昌华嘉股份有限公司等。因此，Document 1提供了关于112年藥品進貨交易对象的信息。

接下来，我检查了Document 2和Document 3。这两个文档都是“民國 111 年度”的，内容涉及医务收入和成本，包括交易对象如国泰人壽、国泰金控等，但这些与药品进货无关，而是关于保险或其他服务的交易。因此，这两个文件对用户的问题没有帮助，可以忽略。

综上所述，只有Document 1提供了112年藥品進貨交易对象的具体信息。
</think>

Document 1 提供了民國 112 年度藥品進貨交易對象的資訊：

1. 裕利股份有限公司  
2. 台灣大昌華嘉股份有限公司  
3. 久裕企業股份有限公司  
4. 華安藥品股份有限公司  
5. 台湾東洋藥品工業股份有限公司  
6. 台田藥品股份有限公司  
7. 台灣中外製药股份有限公司  
8. 瑞安國際股份有限公司  
9. 大隆興藥品股份有限公司  
10. 元英企業股份有限公司
"""
len(tokenizer.tokenize(text))